In [8]:
import torch
from torch import nn
import matplotlib.pyplot as plt
%matplotlib inline

VOCAB_SIZE = 27
EMB_DIM = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cpu


In [5]:
words = open('data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [6]:
itos = dict(enumerate(sorted(set("".join(words))), start=1))
itos[0] = "."
stoi = {v: k for k, v in itos.items()}

In [7]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
class Net(nn.Module):

    def __init__(
            self,
            vocab_size: int = None,
            emb_dim: int = None,
            n_layers: int = 2,
            num_hidden: int = 100,
            out_size: int = None,
            prod_drop: float = .5
    ):
        super(Net, self).__init__()

        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.RNN(input_size=emb_dim, hidden_size=num_hidden, batch_first=True, dropout=prod_drop)
        self.linear = nn.Linear(num_hidden, out_size)

    def forward(self, x):
        pass

    def init_hidden(self, batch_size):
        """ Initializes hidden state """
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden


In [14]:
emb = nn.Embedding(27, 10)
emb

Embedding(27, 10)

In [21]:
input = torch.tensor([[1, 2, 3], [1, 2, 3]])
emb(input)

tensor([[[-1.1846,  0.8093, -0.6294, -0.6847,  1.0518,  0.4880,  0.0813,
           0.0715,  1.8039,  1.0459],
         [ 0.5422,  0.6548, -2.0778, -1.9846, -0.0064,  1.2745, -0.3909,
          -0.4750,  2.0223, -0.3341],
         [ 0.2896,  0.0347, -0.7623,  0.0263, -0.4926, -1.2940,  0.4946,
          -0.0418, -1.1437,  2.1317]],

        [[-1.1846,  0.8093, -0.6294, -0.6847,  1.0518,  0.4880,  0.0813,
           0.0715,  1.8039,  1.0459],
         [ 0.5422,  0.6548, -2.0778, -1.9846, -0.0064,  1.2745, -0.3909,
          -0.4750,  2.0223, -0.3341],
         [ 0.2896,  0.0347, -0.7623,  0.0263, -0.4926, -1.2940,  0.4946,
          -0.0418, -1.1437,  2.1317]]], grad_fn=<EmbeddingBackward0>)

In [22]:
emb.children()

<generator object Module.children at 0x164e92570>